# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/hamma/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Define Search Terms and File Paths

In [7]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98112'
term = 'pizza'

In [21]:
location.split(',')[0]

'Seattle'

In [29]:
## Specify fodler for saving data
folder = 'DataEnrichment/data-enrichment-wk14-activity-mapping-yelp-api-results/'

os.makedirs(folder, exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
json_file = folder+f"{location.split(',')[0]}-{term}.json"

### Check if Json File exists and Create it if it doesn't

In [31]:
## Check if JSON_FILE exists
file_exist = os.path.isfile(json_file)
## If it does not exist: 
if file_exist == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(json_file)
    
    ## If JSON_FILE included a folder:
    if len(folder) >0:
        # create the folder
        os.makedirs(folder, exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {json_file} not found. Saving empty list to file")
    
    
    ## save the first page of results
    with open(json_file, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {json_file} already exists")

[i] DataEnrichment/data-enrichment-wk14-activity-mapping-yelp-api-results/Seattle-pizza.json not found. Saving empty list to file


### Load JSON FIle and account for previous results

In [43]:
## Load previous results and use len of results for offset
with open(json_file,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 980 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [33]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location = location, term = term)

In [34]:
## How many results total?
results['total']

2100

- Where is the actual data we want to save?

In [35]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/Yge-Xa...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4512,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.61304626911818, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '2001 4th Ave', 'address2': None,...",+12068387388,(206) 838-7388,787.405145
1,1rkgyfwzXG7qSdVZdfRt7g,roccos-seattle,Rocco's,https://s3-media3.fl.yelpcdn.com/bphoto/AWBXTE...,False,https://www.yelp.com/biz/roccos-seattle?adjust...,1769,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.614470434736106, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '2312 2nd Ave', 'address2': '', '...",+12063974210,(206) 397-4210,1203.012359
2,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[delivery, pickup]",$,"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,522.368437
3,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/RA8N1B...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,164,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],$$,"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,7127.855269
4,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,109,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[delivery, pickup]",$$,"{'address1': '601 Summit Ave E', 'address2': '...",,,1189.030990
5,n6BLr6spgjVWqzUwmVx0bg,johnny-mos-pizzeria-seattle,Johnny Mo's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/8IAnDP...,False,https://www.yelp.com/biz/johnny-mos-pizzeria-s...,150,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.65142, 'longitude': -122.32083}","[delivery, pickup]",$$,"{'address1': '3272 Fuhrman Ave E', 'address2':...",+12068226272,(206) 822-6272,4182.305164
6,QVmnZZaejnpRQ7A0gG3Zbg,lupo-seattle-3,Lupo,https://s3-media3.fl.yelpcdn.com/bphoto/-qSKGY...,False,https://www.yelp.com/biz/lupo-seattle-3?adjust...,190,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.6591, 'longitude': -122.35023}","[delivery, pickup]",$$,"{'address1': '4303 Fremont Ave N', 'address2':...",+12065472144,(206) 547-2144,5206.258682
7,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[delivery, pickup]",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1293.942778
8,vTz9X1K3UZWdYdajgEcwPQ,seattle-pizza-and-bar-seattle,Seattle Pizza and Bar,https://s3-media1.fl.yelpcdn.com/bphoto/j5urTA...,False,https://www.yelp.com/biz/seattle-pizza-and-bar...,48,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.616109251924556, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '2801 Western Ave', 'address2': '...",+12064415188,(206) 441-5188,1753.510469
9,XmGjU9gxS5s0CzP4S6-bHw,pizzeria-la-rocca-seattle,Pizzeria La Rocca,https://s3-media1.fl.yelpcdn.com/bphoto/tjHF8A...,False,https://www.yelp.com/biz/pizzeria-la-rocca-sea...,114,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.69009, 'longitude': -122.35495}","[delivery, pickup]",$$,"{'address1': '8404 Greenwood Ave N', 'address2...",+12065567656,(206) 556-7656

In [40]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [41]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']) / results_per_page)

In [42]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(0.2)
        
        ## Read in results in progress file and check the length
        with open(json_file, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=location, term=term,
                                       offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(json_file, 'w') as f:
            json.dump(previous_results, f)
            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/105 [00:00<?, ?it/s]

 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
 [!] ERROR VALIDATION_ERROR: Too many resul

## Open the Final JSON File with Pandas

In [46]:
df = pd.read_json(json_file)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,1rkgyfwzXG7qSdVZdfRt7g,roccos-seattle,Rocco's,https://s3-media3.fl.yelpcdn.com/bphoto/AWBXTE...,False,https://www.yelp.com/biz/roccos-seattle?adjust...,1769,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.614470434736106, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '2312 2nd Ave', 'address2': '', '...",+12063974210,(206) 397-4210,1203.012359
1,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[delivery, pickup]",$,"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,522.368437
2,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/RA8N1B...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,164,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],$$,"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,7127.855269
3,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,109,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[delivery, pickup]",$$,"{'address1': '601 Summit Ave E', 'address2': '...",,,1189.030990
4,n6BLr6spgjVWqzUwmVx0bg,johnny-mos-pizzeria-seattle,Johnny Mo's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/8IAnDP...,False,https://www.yelp.com/biz/johnny-mos-pizzeria-s...,150,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.65142, 'longitude': -122.32083}","[delivery, pickup]",$$,"{'address1': '3272 Fuhrman Ave E', 'address2':...",+12068226272,(206) 822-6272,4182.305164


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
975,jQmDJrTse6RF4jnLEvFZpQ,new-saigon-vietnamese-restaurant-ii-seattle,New Saigon Vietnamese Restaurant II,https://s3-media2.fl.yelpcdn.com/bphoto/CLNpKE...,False,https://www.yelp.com/biz/new-saigon-vietnamese...,301,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",2.5,"{'latitude': 47.6116699, 'longitude': -122.33531}",[delivery],$$,"{'address1': '1529 6th Ave', 'address2': '', '...",+12066230212,(206) 623-0212,479.999087
976,r_5tNcnXtfl6qv1PoDdqRQ,food-truck-friday-redmond,Food Truck Friday,https://s3-media1.fl.yelpcdn.com/bphoto/k33iel...,False,https://www.yelp.com/biz/food-truck-friday-red...,7,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 47.70613, 'longitude': -122.10017}",[],NaN,"{'address1': '11810 179th Pl NE', 'address2': ...",,,20016.309342
977,XL6UtF0GljGKOIG0FLum3g,pcc-community-markets-redmond-redmond,PCC Community Markets - Redmond,https://s3-media2.fl.yelpcdn.com/bphoto/J9FA6k...,False,https://www.yelp.com/biz/pcc-community-markets...,137,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.5,"{'latitude': 47.7016, 'longitude': -122.09339}",[],$$,"{'address1': '11435 Avondale Rd NE', 'address2...",+14252851400,(425) 285-1400,20195.334969
978,oMy_YkyYr8u1_xs_WHkn8w,lady-jaye-seattle,Lady Jaye,https://s3-media2.fl.yelpcdn.com/bphoto/NsQoCb...,False,https://www.yelp.com/biz/lady-jaye-seattle?adj...,176,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 47.56223, 'longitude': -122.38706}",[delivery],$$,"{'address1': '4523 California Ave SW', 'addres...",+12064574029,(206) 457-4029,7199.007770
979,OrDvpwx5GWpJWkWpn_dyQQ,takis-mad-greek-seattle,Taki's Mad Greek,https://s3-media1.fl.yelpcdn.com/bphoto/fplNdy...,False,https://www.yelp.com/biz/takis-mad-greek-seatt...,362,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.0,"{'latitude': 47.6904779, 'longitude': -122.375...","[delivery, pickup]",$$,"{'address1': '1471 NW 85th St', 'address2': No...",+12062979200,(206) 297-9200,9136.321778


In [51]:
# Check for duplicates
df.duplicated(subset='id').sum()

0

In [47]:
## convert the filename to a .csv.gz
csv_file = json_file.replace('.json','.csv.gz')
csv_file

'DataEnrichment/data-enrichment-wk14-activity-mapping-yelp-api-results/Seattle-pizza.csv.gz'

In [52]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [54]:
size_json = os.path.getsize(json_file)
size_csv_gz = os.path.getsize(json_file.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 957,469 Bytes
CSV.GZ FILE: 143,238 Bytes
the csv.gz is 6.684462223711585 times smaller!


## Next Class: Processing the Results and Mapping 